# X2Mesh

A user interface for stylizing 3D objects given a text prompt or an image. The interface comes with several models which tests can be ran on, running the imports cell will reveal which models are available for testing
<br />
Based on the work of [text2mesh](https://threedle.github.io/text2mesh/), the code has been modified to create a testing, parameter tuning, and visualizing pipeline. 
<br />
All relevant code for this notebook can be found on the [IMAD](https://github.com/ATKatary/IMAD) github

#### Imports

In [2]:
import shlex
import subprocess
%matplotlib inline
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from implementation.helpers.general_helpers import *

make_prompt = lambda content, style: [f"A {content} made of {style} ",
                                      f"A {style}-shaped {content}",
                                      f"{content} that looks like its made of {style}",
                                      f"{content} in the style of {style}",
                                      f"{content} in the style of beautiful {style}",
                                      f"An artistic {content} that mimics beautiful {style}"]
home_dir = "./implementation"
text2mesh_path = "./text2mesh"
models_dir = f"{home_dir}/inputs/vase/vase_1"
# models_dir = "/home/ubuntu/imad/segmentation/models/vases"
models = get_valid_models(models_dir, models={}, text2mesh_path=text2mesh_path)

Success[./implementation/inputs/vase/vase_1/vase.obj is goo!]: Number of verticies (21311) == the number of normals (21311)
Found 1 working models in ./implementation/inputs/vase/vase_1


## Image2Mesh Testing
#### Prameters

In [ ]:
img_n_aug = 5                                                                  # number of augmentations we add to the image
n_iter = 1000                                                                  # number of iterations to run the network for
img_lr = 0.0005                                                                # learning rate
img_sigma = 10.0                                                               # frequency of texture spreading across mesh
img_lr_decay = 0.9                                                             # decay learning rate
obj, obj_path = select_model(models)
output_dir = "./implementation/outputs/img2mesh/lamp/wood/wood1"
img_path = download(home_dir, fn="style_image")

In [ ]:
!bash ./implementation/img2mesh.sh {obj_path} {output_dir} '{img_path}' {n_iter + 1} {text2mesh_path} {home_dir} {img_sigma} {img_lr_decay} {img_lr} {img_n_aug}

#### Displaying Results

In [ ]:
before_image_path = f"{output_dir}/{obj}_iters_style_5/iter_0.jpg"
after_image_path = f"{output_dir}/{obj}_iters_style_5/iter_{(n_iter // 100) * 100}.jpg"

display([before_image_path, after_image_path])

## Text2Mesh Testing
#### Parameters

In [3]:
run_all = "n"; style = "wood"; content = "vase"
if run_all is None: run_all = input("Run all? (y/n): ")
if style is None: style = input("What is the style that I should test? ")
if content is None: content = input("What is the content that I should test? ")
prompts = make_prompt(content, style)

n_iter = 5 
text_n_aug = 1
text_lr = 0.0005
text_sigma = 10.0
text_lr_decay = 0.9
output_dir = "./implementation/outputs/text2mesh/vase/wood"
vertices_to_not_change = "./implementation/inputs/vase/vase_1/min_y_vertices.txt"
if run_all == "n": 
    obj, obj_path = select_model(models, selected_model="vase")

The available models are:
['vase']


In [4]:
!bash ./implementation/text2mesh.sh {obj_path} {output_dir} '{prompts[0]}' {n_iter + 1} {text2mesh_path} {home_dir} {text_sigma} {text_lr_decay} {text_lr} {text_n_aug} {vertices_to_not_change}

Success[./implementation/inputs/vase/vase_1/vase.obj is goo!]: Number of verticies (21311) == the number of normals (21311)
Saving to ./implementation/outputs/text2mesh/vase/wood/vase_final_style_24 ...
Saving to ./implementation/outputs/text2mesh/vase/wood/vase_iters_style_24 ...
ModuleList(
  (0): FourierFeatureTransform()
  (1): Linear(in_features=515, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): Linear(in_features=256, out_features=256, bias=True)
  (6): ReLU()
  (7): Linear(in_features=256, out_features=256, bias=True)
  (8): ReLU()
  (9): Linear(in_features=256, out_features=256, bias=True)
  (10): ReLU()
)
ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=3, bias=True)
)
ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): R

[light_shape] >> torch.Size([1, 1, 224, 224])
[image] >> torch.Size([1, 224, 224, 3])
[images] >> torch.Size([5, 3, 224, 224])
torch.Size([5, 3, 224, 224]) torch.Size([1, 512])
[light_shape] >> torch.Size([1, 1, 224, 224])
[image] >> torch.Size([1, 224, 224, 3])
[light_shape] >> torch.Size([1, 1, 224, 224])
[image] >> torch.Size([1, 224, 224, 3])
[light_shape] >> torch.Size([1, 1, 224, 224])
[image] >> torch.Size([1, 224, 224, 3])
[light_shape] >> torch.Size([1, 1, 224, 224])
[image] >> torch.Size([1, 224, 224, 3])
[light_shape] >> torch.Size([1, 1, 224, 224])
[image] >> torch.Size([1, 224, 224, 3])
[images] >> torch.Size([5, 3, 224, 224])
[loss] >> tensor([-0.9727], device='cuda:0', dtype=torch.float16, grad_fn=<SubBackward0>)
[norm_loss] >> tensor([-1.6279], device='cuda:0', dtype=torch.float16, grad_fn=<SubBackward0>)
100%|█████████████████████████████████████████████| 6/6 [00:09<00:00,  1.58s/it]
[light_shape] >> torch.Size([1, 1, 720, 1280])
[image] >> torch.Size([1, 720, 1280, 3]

#### Batch Testing

In [ ]:
if run_all == "y":
    for obj, obj_path in models.items():
        for prompt in prompts:
            subprocess.call(shlex.split(f"bash ./implementation/img2mesh.sh {obj_path} {output_dir} '{prompt}' {n_iter + 1} {text2mesh_path} {home_dir} {text_sigma} {text_lr_decay} {text_lr} {text_n_aug}"))
else: 
    for prompt in prompts:
        subprocess.call(shlex.split(f"bash ./implementation/img2mesh.sh {obj_path} {output_dir} '{prompt}' {n_iter + 1} {text2mesh_path} {home_dir} {text_sigma} {text_lr_decay} {text_lr} {text_n_aug}"))

#### Display Results

In [ ]:
before_image_path = f"{output_dir}/{obj}_iters_style_0/iter_0.jpg"
after_image_path = f"{output_dir}/{obj}_iters_style_0/iter_{(n_iter // 100) * 100}.jpg"

display([before_image_path, after_image_path])

## Attributes
The original [text2mesh](https://github.com/threedle/text2mesh) repositotory was cloned in the creation of this notebook  